# 월간 데이콘 2 천체 유형 분류
## Public 8위 Private 10위 코드 

2주~3주간 처음으로 참여한 천체관측 대회가 마무리 됐네요.
모두 수고 많으셨고 별거 아니지만 대회 진행과정 공유합니다.

저도 못했지만 캐글처럼 활발한 지식 공유가 이루어지면 좋겠습니다.
다른분들도 후기 남겨주시면 보고 배우겠습니다 ㅎㅎ

모델
처음에는 lgbm, xgboost, catboost 세가지의 모델의 결과의 앙상블을 통해 순위를 조금씩 올렸습니다. 

최종적으로는 lgbm 단일모델로 하이퍼 파라미터를 조율하며 마무리했습니다.

그리고 다중 분류 방법 중 one verse all 기법을 이용해보고 싶었는데 좋은 자료를 찾지못해 포기한점이 아쉽습니다.
lgbm에서는 objective = 'multicalssova'을 이용하면 쉽게 사용할 수 있는데 제경우에는 overfitting이 심해져 포기했습니다.

변수 
초반에는 fiberID를 이용해 1000개의 모델을 만들어보려고 했는데 학습 데이터를 나누면 train label과 validation label이 달라 학습되지 않는 부분이 해결되지 않아 포기했습니다.
validation 없이 진행 해서 적용해보았지만 결과가 좋지 못했습니다.
하지만 이과정에서 QSO만 관측한 fiberID를 찾아낼 수 있었고 이 fiberID들의 결과를 모두 QSO로 확정한것이 미세하게 점수를 올리는데 도움된것 같습니다.

FEATURE Engineering
파장별, 관측법 별 mean, std를 구하고 변수별로 pca를 적용했습니다. 
mean과 std에도 pca를 적용했는데 std의 pca 변수가 모델의 feature importance의 상위권에 올라져 있는 이유를 아직도 모르겠습니다.

일주일을 채 남겨두지 않고, 토론글을 통해 도메인 지식을 이용해 새로운 변수를 생성할 수 있었습니다. 

하이퍼 파라미터
하이퍼 파라미터는 베이지안 옵티마이제이션을 이용해 적절한 변수를 찾고 최종적으로 num_leaves와 max_depth,  learning_rate를 조절하며 최선의 하이퍼파라미터를 찾았습니다.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
import math 

from sklearn.ensemble import IsolationForest

In [ ]:
train = pd.read_csv('../data/train.csv', index_col=0)
test = pd.read_csv('../data/test.csv', index_col=0)
sample_submission = pd.read_csv('../data/sample_submission.csv', index_col=0)

Feature engineering  
FiberID 별로 관측된 천체 타입의 종류가 다름을 인지하고 이에 따른 Typenum 변수 생성
특히 640? 이후로는 모든 변수가 QSO임을 확인하고 이를 제외하고 train하기로 마음먹음

In [ ]:
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x: to_number(x, column_number))

In [ ]:
# Make Fiber tpye
fiber = []
label_dict = {}

for i in list(set(train['fiberID'])):
    train2 = train[train['fiberID']==i]
    #test2 = test[test['fiberID']==i]
    #idx2 = test2.index
    idx1 = train2.index
    #print(set(train.loc[idx1]['type_num']))
    if tuple(set(train.loc[idx1]['type_num'])) not in fiber:
        fiber.append(tuple(set(train.loc[idx1]['type_num'])))
    label_dict[i] = list(set(train.loc[idx1]['type_num']))
    
    
fiber_type = {}
for k in range(len(fiber)):
    fiber_type[fiber[k]] = k
    
fiber_dict = {}  
for j in list(set(train['fiberID'])):
    train2 = train[train['fiberID']==j]
    #test2 = test[test['fiberID']==i]
    idx1 = train2.index
    
    #print(set(train.loc[idx1]['type_num']))
    fiber_dict[j] = fiber_type[tuple(set(train.loc[idx1]['type_num']))]

In [ ]:
ntrain = train.shape[0]
ntest = test.shape[0]
all_data = train.append(test)

In [ ]:
all_data['fiberType'] = all_data['fiberID']
Fill_Type = lambda x: fiber_dict[x]
all_data['fiberType'] = all_data['fiberType'].apply(Fill_Type)

In [ ]:
psfMags = ['psfMag_u','psfMag_g','psfMag_r','psfMag_i','psfMag_z']
fiberMags = ['fiberMag_u','fiberMag_g','fiberMag_r','fiberMag_i','fiberMag_z']
petroMags = ['petroMag_u','petroMag_g','petroMag_r','petroMag_i','petroMag_z']
modelMags = ['modelMag_u','modelMag_g','modelMag_r','modelMag_i','modelMag_z']

In [ ]:
u_values = ['psfMag_u','fiberMag_u','petroMag_u','modelMag_u']
g_values = ['psfMag_g','fiberMag_g','petroMag_g','modelMag_g']
r_values = ['psfMag_r','fiberMag_r','petroMag_r','modelMag_r']
i_values = ['psfMag_i','fiberMag_i','petroMag_i','modelMag_i']
z_values = ['psfMag_z','fiberMag_z','petroMag_z','modelMag_z']

파장 변수 Engineering  
강경수님의 토론 글을 읽고 구글링을 통해 u-r,u-g,g-r,g-z,g-i 등의 변수가 천체 분류에 효과적이라는 사실을 알아냄 50위권>10위권 상승  
관련 자료  
http://dspace.ewha.ac.kr/handle/2015.oak/211793  
http://www.koreascience.or.kr/article/CFKO201023859828644.pub  
*** 기타 변수들은 임의로 만들어봄  

In [ ]:
u_r = ['psfMag_u-r', 'fiberMag_u-r','petroMag_u-r','modelMag_u-r']
u_g = ['psfMag_u-g','fiberMag_u-g',  'petroMag_u-g','modelMag_u-g']
g_r = ['psfMag_g-r', 'fiberMag_g-r',  'petroMag_g-r', 'modelMag_g-r']      
g_z = ['psfMag_g-z','fiberMag_g-z', 'petroMag_g-z','modelMag_g-z'] 
g_i = ['psfMag_g-i','fiberMag_g-i','petroMag_g-i', 'modelMag_g-i']

u_i = ['psfMag_u-i','fiberMag_u-i','petroMag_u-i', 'modelMag_u-i']
u_z = ['psfMag_u-z','fiberMag_u-z','petroMag_u-z', 'modelMag_u-z']
r_i = ['psfMag_r-i','fiberMag_r-i','petroMag_r-i', 'modelMag_r-i']
r_z = ['psfMag_r-z','fiberMag_r-z','petroMag_r-z', 'modelMag_r-z']
i_z = ['psfMag_i-z','fiberMag_i-z','petroMag_i-z', 'modelMag_i-z']

In [ ]:
u_r = ['psfMag_u-r', 'fiberMag_u-r','petroMag_u-r','modelMag_u-r']
u_g = ['psfMag_u-g','fiberMag_u-g',  'petroMag_u-g','modelMag_u-g']
g_r = ['psfMag_g-r', 'fiberMag_g-r',  'petroMag_g-r', 'modelMag_g-r']      
g_z = ['psfMag_g-z','fiberMag_g-z', 'petroMag_g-z','modelMag_g-z'] 
g_i = ['psfMag_g-i','fiberMag_g-i','petroMag_g-i', 'modelMag_g-i']

u_i = ['psfMag_u-i','fiberMag_u-i','petroMag_u-i', 'modelMag_u-i']
u_z = ['psfMag_u-z','fiberMag_u-z','petroMag_u-z', 'modelMag_u-z']
r_i = ['psfMag_r-i','fiberMag_r-i','petroMag_r-i', 'modelMag_r-i']
r_z = ['psfMag_r-z','fiberMag_r-z','petroMag_r-z', 'modelMag_r-z']
i_z = ['psfMag_i-z','fiberMag_i-z','petroMag_i-z', 'modelMag_i-z']

sumur = ['psfMag_u+r', 'fiberMag_u+r','petroMag_u+r','modelMag_u+r']
sumug = ['psfMag_u+g','fiberMag_u+g',  'petroMag_u+g','modelMag_u+g']
sumgr = ['psfMag_g+r', 'fiberMag_g+r',  'petroMag_g+r', 'modelMag_g+r']      
sumgz = ['psfMag_g+z','fiberMag_g+z', 'petroMag_g+z','modelMag_g+z'] 
sumgi = ['psfMag_g+i','fiberMag_g+i','petroMag_g+i', 'modelMag_g+i']

sumui = ['psfMag_u+i','fiberMag_u+i','petroMag_u+i', 'modelMag_u+i']
sumuz = ['psfMag_u+z','fiberMag_u+z','petroMag_u+z', 'modelMag_u+z']
sumri = ['psfMag_r+i','fiberMag_r+i','petroMag_r+i', 'modelMag_r+i']
sumrz = ['psfMag_r+z','fiberMag_r+z','petroMag_r+z', 'modelMag_r+z']
sumiz = ['psfMag_i+z','fiberMag_i+z','petroMag_i+z', 'modelMag_i+z']

In [ ]:
cols = all_data.columns.drop(['type','type_num','fiberType','fiberID'])
pca = PCA(n_components = 2)

x = all_data[cols].values
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['all_pca1','all_pca2'])
all_data = pd.concat([all_data,principalDf],axis =1)

In [ ]:
for i in range(4):
    all_data[str(u_values[i])+'-'+'r'] = all_data[u_values[i]]-all_data[r_values[i]]
    all_data[str(u_values[i])+'-'+'g'] = all_data[u_values[i]]-all_data[g_values[i]]
    all_data[str(g_values[i])+'-'+'r'] = all_data[g_values[i]]-all_data[r_values[i]]
    all_data[str(g_values[i])+'-'+'z'] = all_data[g_values[i]]-all_data[z_values[i]]
    all_data[str(g_values[i])+'-'+'i'] = all_data[g_values[i]]-all_data[i_values[i]]
    
    all_data[str(u_values[i])+'-'+'i'] = all_data[u_values[i]]-all_data[i_values[i]]
    all_data[str(u_values[i])+'-'+'z'] = all_data[u_values[i]]-all_data[z_values[i]]
    all_data[str(r_values[i])+'-'+'z'] = all_data[r_values[i]]-all_data[z_values[i]]
    all_data[str(r_values[i])+'-'+'i'] = all_data[r_values[i]]-all_data[i_values[i]]
    all_data[str(i_values[i])+'-'+'z'] = all_data[i_values[i]]-all_data[z_values[i]]
        
        
        
    all_data[str(u_values[i])+'+'+'r'] = all_data[u_values[i]]+all_data[r_values[i]]
    all_data[str(u_values[i])+'+'+'g'] = all_data[u_values[i]]+all_data[g_values[i]]
    all_data[str(g_values[i])+'+'+'r'] = all_data[g_values[i]]+all_data[r_values[i]]
    all_data[str(g_values[i])+'+'+'z'] = all_data[g_values[i]]+all_data[z_values[i]]
    all_data[str(g_values[i])+'+'+'i'] = all_data[g_values[i]]+all_data[i_values[i]]
    
    all_data[str(u_values[i])+'+'+'i'] = all_data[u_values[i]]+all_data[i_values[i]]
    all_data[str(u_values[i])+'+'+'z'] = all_data[u_values[i]]+all_data[z_values[i]]
    all_data[str(r_values[i])+'+'+'z'] = all_data[r_values[i]]+all_data[z_values[i]]
    all_data[str(r_values[i])+'+'+'i'] = all_data[r_values[i]]+all_data[i_values[i]]
    all_data[str(i_values[i])+'+'+'z'] = all_data[i_values[i]]+all_data[z_values[i]]


In [ ]:
val_groups = [psfMags,fiberMags,petroMags,modelMags,u_values,g_values,r_values,i_values,z_values,u_r,u_g,g_r,g_z,g_i,u_i,u_z,r_i,r_z,i_z,sumur, sumug, sumgr,  sumgz, sumgi, sumui, sumuz, sumri ,sumrz ,sumiz]
group_names = ['psf','fiber','petro','model','u','g','r','i','z','u-r','u-g','g-r','g-z','g-i','u-i','u-z','r-i','r-z','i-z','sumur', 'sumug', 'sumgr',  'sumgz', 'sumgi', 'sumui', 'sumuz', 'sumri' ,'sumrz' ,'sumiz']

In [ ]:
pca = PCA(n_components = 3)

for i in range(len(val_groups)):
    
    all_data[str(group_names[i])+'_mean'] = all_data[val_groups[i]].mean(axis =1)
    all_data[str(group_names[i])+'_std'] = all_data[val_groups[i]].std(axis =1)

    x = all_data[val_groups[i]].values
    principalComponents = pca.fit_transform(x)
    principalDf = pd.DataFrame(data = principalComponents
             , columns = [str(group_names[i]) + '_pca1', str(group_names[i]) + '_pca2',str(group_names[i]) + '_pca3'])
    #print(group_names[i])
    #print('explained variance ratio :', pca.explained_variance_ratio_)

    
    
    all_data = pd.concat([all_data,principalDf],axis =1)

트리 구조 특성상 더 잘 학습시키기 위해 45 Rotation한 변수 생성  
***행렬 곱을 이용해 더 효과적인 코드를 짰지만 무슨 이유인지 성능이 좋지 않아 이 방법을 씀

In [ ]:
def RotateXY(x,y,xc=0,yc=0,angle=0,units="DEGREES"):  
    """Rotate an xy cooordinate about a specified origin  
  
    x,y      xy coordinates  
    xc,yc   center of rotation  
    angle   angle  
    units    "DEGREES" (default) or "RADIANS"  
    """  
    import math  
    x = x - xc  
    y = y - yc  
    if units == "DEGREES":  
        angle = math.radians(angle)  
    xr = (x * math.cos(angle)) - (y * math.sin(angle)) + xc  
    yr = (x * math.sin(angle)) + (y * math.cos(angle)) + yc  
    return xr, yr  

In [ ]:
import datetime
for i in range(4):
    start = datetime.datetime.now()
    u_g_1 = []
    u_g_2 = []
    g_r_1 = []
    g_r_2 = []
    g_z_1 = []
    g_z_2 = []
    g_i_1 = []
    g_i_2 = []
    r_i_1 = []
    r_i_2 = []
        
    for k in range(len(all_data)):
        
        Rotate = RotateXY(all_data[str(u_values[i])+'-'+'g'].loc[k],all_data[str(r_values[i])].loc[k],xc=0,yc=0,angle=45,units="DEGREES")
        u_g_1.append(Rotate[0])
        u_g_2.append(Rotate[1])
        
        Rotate = RotateXY(all_data[str(g_values[i])+'-'+'r'].loc[k],all_data[str(r_values[i])].loc[k],xc=0,yc=0,angle=45,units="DEGREES")
        g_r_1.append(Rotate[0])
        g_r_2.append(Rotate[1])
        
        
        Rotate = RotateXY(all_data[str(g_values[i])+'-'+'z'].loc[k],all_data[str(r_values[i])].loc[k],xc=0,yc=0,angle=45,units="DEGREES")
        g_z_1.append(Rotate[0])
        g_z_2.append(Rotate[1])
        
        Rotate = RotateXY(all_data[str(g_values[i])+'-'+'i'].loc[k],all_data[str(r_values[i])].loc[k],xc=0,yc=0,angle=45,units="DEGREES")
        g_i_1.append(Rotate[0])
        g_i_2.append(Rotate[1])
        
        Rotate = RotateXY(all_data[str(r_values[i])+'-'+'i'] .loc[k],all_data[str(r_values[i])].loc[k],xc=0,yc=0,angle=45,units="DEGREES")
        r_i_1.append(Rotate[0])
        r_i_2.append(Rotate[1])   
        
    ug1= pd.DataFrame(data = u_g_1
             , columns = [str(u_values[i])+'-'+'g'+'r451'])
    ug2= pd.DataFrame(data = u_g_2
             , columns = [str(u_values[i])+'-'+'g'+'r452'])
       
    gr1= pd.DataFrame(data = g_r_1
             , columns = [str(g_values[i])+'-'+'r'+'r451'])
    gr2= pd.DataFrame(data = g_r_2
             , columns = [str(g_values[i])+'-'+'r'+'r452'])
        
    gz1= pd.DataFrame(data = g_z_1
             , columns = [str(g_values[i])+'-'+'z'+'r451'])
    gz2= pd.DataFrame(data = g_z_2
             , columns = [str(g_values[i])+'-'+'z'+'r452'])
        
        
    gi1= pd.DataFrame(data = g_i_1
             , columns = [str(g_values[i])+'-'+'i'+'r451'])
    gi2= pd.DataFrame(data = g_i_2
             , columns = [str(g_values[i])+'-'+'i'+'r452'])   
    
    ri1= pd.DataFrame(data = r_i_1
             , columns = [str(r_values[i])+'-'+'i'+'r451'])
    ri2= pd.DataFrame(data = r_i_2
             , columns = [str(r_values[i])+'-'+'i'+'r452'])
             
    
    all_data = pd.concat([all_data,ug1,ug2,gr1,gr2,gz1,gz2,gi1,gi1,ri1,ri2],axis =1)
    runtime = datetime.datetime.now() - start
    break    

In [ ]:
train= all_data[:ntrain]
test= all_data[ntrain:]

Train set에서 FiberType ==34인것 제외  
output이 모두 QSO인 FiberID들은 train에서 제외하고 추후에 QSO 확률 1,나머지 0으로 채워넣음

In [ ]:
y = train['type_num']
train = train.drop(columns=['type','type_num'], axis=1)
train1 = train[train['fiberType'] != 34]
train2 = train[train['fiberType'] == 34]
train1_idx = train1.index 
train2_idx = train2.index
y1 = y.loc[train1_idx]

test = test.drop(['type','type_num'], axis=1)
test1 = test[test['fiberType'] != 34]
test2 = test[test['fiberType'] == 34]
test1_idx = test1.index
test2_idx = test2.index

In [ ]:
from sklearn.model_selection import StratifiedKFold

kfold = 20
kf = StratifiedKFold(n_splits=kfold, shuffle=True, random_state = 42)
preds =[]
for train_index, test_index in kf.split(train1, y1):
    x_train, x_val = train1.iloc[train_index], train1.iloc[test_index]
    y_train, y_val = y1.iloc[train_index], y1.iloc[test_index]
    
    clf = lgb.LGBMClassifier(n_estimators = 10000, max_depth=10, learning_rate=0.009, objective='multiclass',num_class = 19
                             ,num_leaves = 1024,random_state=42,metric = 'multi_logloss',boost_from_average = False,
                            feature_fraction =0.21815809919426804,bagging_fraction = 0.8055711236002633,lambda_l1 = 0.0029896841778409566,
                             lambda_l2 = 0.06905300928232105,min_split_gain = 0.05931126989247223,min_child_weight = 10)
    clf.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_val, y_val)],
            early_stopping_rounds=50, verbose=100)
    
    
    
    # fiberID ==34 인 test_set에 QSO =1로 채워넣음
    ps = pd.DataFrame(data=clf.predict_proba(test), index=test.index)
    sam = sample_submission.copy()
    sam.loc[test1_idx] =ps.loc[test1_idx]
    sam.loc[test2_idx] = 0
    sam.loc[test2_idx,'QSO'] = 1
    

    preds.append(sam)

In [ ]:
pred = sum(preds)/20
pred.index = sample_submission.index
pred.columns = sample_submission.columns
pred.to_csv('LGBM_20_Final.csv', index=True)

하이퍼옵트 라이브러리 이용하고
max_depth, num_leaves, learning_rate는 후보군을 만들어서  돌려봤습니다.